# Identifying key genes for network analysis

This script identifies the key genes through various network centrality measures. It takes as input four lists of differentially expressed genes, and scores them based on centrality measures.

In [3]:
import networkx as nx
import pandas as pd
# import scipy
import random
import numpy as np

## Network measure calculations

This block is no longer needed.

In [ ]:
# ========== LOAD PPI NETWORK ==========
ppi_df = pd.read_csv("../data/networks/combined_PPI.csv")
ppi_graph = nx.from_pandas_edgelist(ppi_df, source="GeneA", target="GeneB")

print(f"PPI graph has {ppi_graph.number_of_nodes()} nodes and {ppi_graph.number_of_edges()} edges.")


# ========== LOAD DEG LIST ==========
deg_df = pd.read_csv("../data/networks/step1_deg.csv", header=None)
deg_list_raw = deg_df.iloc[:, 0].dropna().unique().tolist()

# Filter to DEGs present in the PPI graph
deg_list = [gene for gene in deg_list_raw if gene in ppi_graph.nodes]

print(f"Original DEG list: {len(deg_list_raw)} genes")
print(f"DEGs in PPI network: {len(deg_list)} genes")


# ========== COMPUTE DEGREE CENTRALITY ==========
ppi_graph_nodes = ppi_graph.nodes()

try:
    degree_centrality_dict = nx.degree_centrality(ppi_graph)
except:
    degree_centrality_dict = dict()
    for node in ppi_graph_nodes:
        degree_centrality_dict[node] = 0.0

# Keep only degree centrality for DEGs
deg_degree_centrality = {gene: degree_centrality_dict[gene] for gene in deg_list}

# Convert to DataFrame
deg_degree_centrality_df = pd.DataFrame.from_dict(deg_degree_centrality, orient='index', columns=['DegreeCentrality'])
deg_degree_centrality_df.index.name = "Gene"
deg_degree_centrality_df.reset_index(inplace=True)

# Preview
print(deg_degree_centrality_df.head())


# ========== COMPUTE EIGENVECTOR CENTRALITY ==========
try:
    eigen_centrality_dict = nx.eigenvector_centrality(ppi_graph)
except:
    eigen_centrality_dict = dict()
    for node in ppi_graph_nodes:
        eigen_centrality_dict[node] = 0.0

# Keep only eigenvector centrality for DEGs
deg_eigen_centrality = {gene: eigen_centrality_dict[gene] for gene in deg_list}

# Convert to DataFrame
deg_eigen_centrality_df = pd.DataFrame.from_dict(deg_eigen_centrality, orient='index', columns=['EigenvectorCentrality'])
deg_eigen_centrality_df.index.name = "Gene"
deg_eigen_centrality_df.reset_index(inplace=True)

# Preview
print(deg_eigen_centrality_df.head())


# ========== COMPUTE BETWEENNESS CENTRALITY ==========
try:
    betw_centrality_dict = nx.betweenness_centrality_subset(ppi_graph, sources=deg_list, targets=deg_list)
except:
    betw_centrality_dict = dict()
    for node in ppi_graph_nodes:
        betw_centrality_dict[node] = 0.0

# Keep only eigenvector centrality for DEGs
deg_betw_centrality = {gene: betw_centrality_dict[gene] for gene in deg_list}

# Convert to DataFrame
deg_betw_centrality_df = pd.DataFrame.from_dict(deg_betw_centrality, orient='index', columns=['BetweennessCentrality'])
deg_betw_centrality_df.index.name = "Gene"
deg_betw_centrality_df.reset_index(inplace=True)

# Preview
print(deg_betw_centrality_df.head())


# ========== COMPUTE PERSONALISED PAGERANK ==========
personalization = {gene: 1/len(deg_list) for gene in deg_list} # The walker is equally likely to start at any DEG
page_rank = nx.pagerank(ppi_graph, personalization=personalization, alpha=0.85)

# Keep only PageRank for DEGs
deg_page_rank = {gene: page_rank[gene] for gene in deg_list}

# Convert to DataFrame
deg_page_rank_df = pd.DataFrame.from_dict(deg_page_rank, orient='index', columns=['PageRank'])
deg_page_rank_df.index.name = "Gene"
deg_page_rank_df.reset_index(inplace=True)

# Preview
print(deg_page_rank_df.head())


# ========== COMBINE ALL CENTRALITY SCORES ==========
network_property_df = pd.DataFrame(columns=["Degree", "Eigen", "Between", "PageRank"])

for gene in deg_list:
    network_property_df.loc[gene] = [
        degree_centrality_dict[gene],
        eigen_centrality_dict[gene],
        betw_centrality_dict[gene],
        page_rank[gene],
    ]

# Reset index to make 'Gene' a column
network_property_df.index.name = "Gene"
network_property_df.reset_index(inplace=True)

# Preview
print(network_property_df.head())

# Save
network_property_df.to_csv("../results/humanPVATsn/network_analysis/step1_centrality_scores.csv", index=False)


## Permutation testing

This block is no longer needed. It has been incorporated into the function below.

In [ ]:
# Number of permutations
n_permutations = 50            # Increase to 1000 for more robust results

# Precompute centrality scores for the entire PPI graph
precomputed_degree = nx.degree_centrality(ppi_graph)
precomputed_eigen = nx.eigenvector_centrality(ppi_graph)

# Size of DEG list for step1
deg_list_size = len(deg_list)

# Function to compute centrality scores for a given gene list and graph
def compute_centrality_scores(gene_list, graph, precomputed_degree, precomputed_eigen):
    # Use precomputed values for speed
    degree_scores = [precomputed_degree.get(g, 0.0) for g in gene_list]
    eigen_scores = [precomputed_eigen.get(g, 0.0) for g in gene_list]
    
    # Betweenness
    try:
        betweenness_centrality = nx.betweenness_centrality_subset(graph, sources=gene_list, targets=gene_list)
    except:
        betweenness_centrality = {node: 0.0 for node in graph.nodes}
    between_scores = [betweenness_centrality.get(g, 0.0) for g in gene_list]

    # Personalized PageRank
    personalization = {gene: 1/len(gene_list) for gene in gene_list}
    pagerank = nx.pagerank(graph, personalization=personalization, alpha=0.85)
    pagerank_scores = [pagerank.get(g, 0.0) for g in gene_list]

    return {
        "degree": degree_scores,
        "eigen": eigen_scores,
        "between": between_scores,
        "pagerank": pagerank_scores
    }

# Generate n_permutations random DEG-like gene lists (same length, sampled from graph)
random_gene_lists = []
for i in range(n_permutations):
    random_genes = random.sample(list(ppi_graph.nodes()), deg_list_size)
    random_gene_lists.append(random_genes)

# Initialise global null distributions
null_distributions = {
    "degree": [],
    "eigen": [],
    "between": [],
    "pagerank": []
}

# Populate null distributions across permutations
for i, random_genes in enumerate(random_gene_lists):
    scores = compute_centrality_scores(random_genes, ppi_graph, precomputed_degree=precomputed_degree, precomputed_eigen=precomputed_eigen)

    null_distributions["degree"].extend(scores["degree"])
    null_distributions["eigen"].extend(scores["eigen"])
    null_distributions["between"].extend(scores["between"])
    null_distributions["pagerank"].extend(scores["pagerank"])

    if (i + 1) % 10 == 0:
        print(f"{i+1}/{n_permutations} permutations complete.")

# Compute observed centrality scores for the original DEG list
observed_scores = compute_centrality_scores(deg_list, ppi_graph, precomputed_degree=precomputed_degree, precomputed_eigen=precomputed_eigen)

# Compute empirical p-values for each gene and centrality measure
gene_pvals = {
    "Gene": [],
    "p_degree": [],
    "p_eigen": [],
    "p_between": [],
    "p_pagerank": []
}

for i, gene in enumerate(deg_list):
    gene_pvals["Gene"].append(gene)
    gene_pvals["p_degree"].append(
        np.mean(np.array(null_distributions["degree"]) >= observed_scores["degree"][i])
    )
    gene_pvals["p_eigen"].append(
        np.mean(np.array(null_distributions["eigen"]) >= observed_scores["eigen"][i])
    )
    gene_pvals["p_between"].append(
        np.mean(np.array(null_distributions["between"]) >= observed_scores["between"][i])
    )
    gene_pvals["p_pagerank"].append(
        np.mean(np.array(null_distributions["pagerank"]) >= observed_scores["pagerank"][i])
    )

# Convert to dataframe
gene_pvals_df = pd.DataFrame(gene_pvals)
# print(gene_pvals_df.head())
# print(gene_pvals_df.describe())

# Get number of significant genes and names
significant_genes = gene_pvals_df[
    (gene_pvals_df["p_degree"] <= 0.05) |
    (gene_pvals_df["p_eigen"] <= 0.05) |
    (gene_pvals_df["p_between"] <= 0.05) |
    (gene_pvals_df["p_pagerank"] <= 0.05)
]
n_significant = (gene_pvals_df[["p_degree", "p_eigen", "p_between", "p_pagerank"]] <= 0.05).any(axis=1).sum()

# Print significant genes
print(f"{n_significant} genes have at least one p-value ≤ 0.05 for step 1")
print("Significant genes for step 1 (p ≤ 0.05):")
print(significant_genes["Gene"].tolist())

## Full pipeline to identify key genes for each input list

In [1]:
import networkx as nx
import pandas as pd
import scipy
import random
import numpy as np
import time

In [ ]:
def compute_centrality_scores(gene_list, graph, precomputed_degree, precomputed_eigen):
    
    # Use precomputed values for efficiency
    degree_scores = [precomputed_degree.get(g, 0.0) for g in gene_list]
    eigen_scores = [precomputed_eigen.get(g, 0.0) for g in gene_list]
    
    # Betweenness centrality
    try:
        betweenness_centrality = nx.betweenness_centrality_subset(graph, sources=gene_list, targets=gene_list)
    except:
        betweenness_centrality = {node: 0.0 for node in graph.nodes}
    between_scores = [betweenness_centrality.get(g, 0.0) for g in gene_list]

    # Personalized PageRank
    personalization = {gene: 1/len(gene_list) for gene in gene_list}
    pagerank = nx.pagerank(graph, personalization=personalization, alpha=0.85)
    pagerank_scores = [pagerank.get(g, 0.0) for g in gene_list]

    return {
        "degree": degree_scores,
        "eigen": eigen_scores,
        "between": between_scores,
        "pagerank": pagerank_scores
    }


def identify_key_genes(deg_path, ppi_graph, n_permutations=100, step_label="step"):
    
    print("\nIdentifying key genes for", step_label, "...\n")

    # ========== LOAD DEG LIST ==========
    deg_df = pd.read_csv(deg_path, header=None)
    deg_list_raw = deg_df.iloc[:, 0].dropna().unique().tolist()

    # Filter to DEGs present in the PPI graph
    deg_list = [gene for gene in deg_list_raw if gene in ppi_graph.nodes]

    print(f"Original DEG list for {step_label}: {len(deg_list_raw)} genes")
    print(f"DEGs in PPI network for {step_label}: {len(deg_list)} genes")

    # ========== PERMUTATION TESTING ==========

    # Precompute centrality scores for entire PPI graph
    precomputed_degree = nx.degree_centrality(ppi_graph)
    precomputed_eigen = nx.eigenvector_centrality(ppi_graph)

    # Size of DEG list for step1
    deg_list_size = len(deg_list)

    # Generate n_permutations random DEG-like gene lists (same length, sampled from graph)
    random_gene_lists = []
    for i in range(n_permutations):
        random_genes = random.sample(list(ppi_graph.nodes()), deg_list_size)
        random_gene_lists.append(random_genes)

    # Initialise global null distributions
    null_distributions = {
        "degree": [],
        "eigen": [],
        "between": [],
        "pagerank": []
    }

    # Populate null distributions across permutations
    for i, random_genes in enumerate(random_gene_lists):
        scores = compute_centrality_scores(random_genes, ppi_graph, precomputed_degree=precomputed_degree, precomputed_eigen=precomputed_eigen)

        null_distributions["degree"].extend(scores["degree"])
        null_distributions["eigen"].extend(scores["eigen"])
        null_distributions["between"].extend(scores["between"])
        null_distributions["pagerank"].extend(scores["pagerank"])

        if (i + 1) % 10 == 0:
            print(f"{i+1}/{n_permutations} permutations complete.")

    # ========== COMPUTE OBSERVED CENTRALITY SCORES ==========
    # Compute observed centrality scores for the original DEG list
    observed_scores = compute_centrality_scores(deg_list, ppi_graph, precomputed_degree=precomputed_degree, precomputed_eigen=precomputed_eigen)

    # Compute empirical p-values for each gene and centrality measure
    gene_pvals = {
        "Gene": [],
        "p_degree": [],
        "p_eigen": [],
        "p_between": [],
        "p_pagerank": []
    }
    for i, gene in enumerate(deg_list):
        gene_pvals["Gene"].append(gene)
        gene_pvals["p_degree"].append(
            np.mean(np.array(null_distributions["degree"]) >= observed_scores["degree"][i])
        )
        gene_pvals["p_eigen"].append(
            np.mean(np.array(null_distributions["eigen"]) >= observed_scores["eigen"][i])
        )
        gene_pvals["p_between"].append(
            np.mean(np.array(null_distributions["between"]) >= observed_scores["between"][i])
        )
        gene_pvals["p_pagerank"].append(
            np.mean(np.array(null_distributions["pagerank"]) >= observed_scores["pagerank"][i])
        )

    # Get number of significant genes and names
    gene_pvals_df = pd.DataFrame(gene_pvals)
    significant_genes = gene_pvals_df[
        (gene_pvals_df["p_degree"] <= 0.05) |
        (gene_pvals_df["p_eigen"] <= 0.05) |
        (gene_pvals_df["p_between"] <= 0.05) |
        (gene_pvals_df["p_pagerank"] <= 0.05)
    ]
    n_significant = (gene_pvals_df[["p_degree", "p_eigen", "p_between", "p_pagerank"]] <= 0.05).any(axis=1).sum()

    # Save results
    gene_pvals_df.to_csv(f"../results/humanPVATsn/network_analysis/{step_label}_gene_pvals.csv", index=False)
    significant_genes.to_csv(f"../results/humanPVATsn/network_analysis/{step_label}_significant_genes.csv", index=False)

    # Print significant genes
    print(f"{n_significant} genes have at least one p-value ≤ 0.05 for", step_label)
    print("Significant genes for",  step_label, "(p ≤ 0.05):")
    print(significant_genes["Gene"].tolist())

    return gene_pvals_df, significant_genes

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Load PPI network
ppi_df = pd.read_csv("../data/networks/combined_PPI.csv")
ppi_graph = nx.from_pandas_edgelist(ppi_df, source="GeneA", target="GeneB")
print(f"PPI graph has {ppi_graph.number_of_nodes()} nodes and {ppi_graph.number_of_edges()} edges.")

# Identify key genes for each step
start = time.time()
step1_pvals, step1_key_genes = identify_key_genes("../data/networks/step1_deg.csv", ppi_graph, n_permutations=100, step_label="step1")
print(f"Step 1 done in {(time.time() - start)/60:.2f} minutes.")

start = time.time()
step2_pvals, step2_key_genes = identify_key_genes("../data/networks/step2_deg.csv", ppi_graph, n_permutations=100, step_label="step2")
print(f"Step 2 done in {(time.time() - start)/60:.2f} minutes.")

start = time.time()
step3_pvals, step3_key_genes = identify_key_genes("../data/networks/step3_deg.csv", ppi_graph, n_permutations=100, step_label="step3")
print(f"Step 3 done in {(time.time() - start)/60:.2f} minutes.")

start = time.time()
full_pvals, full_key_genes = identify_key_genes("../data/networks/full_diff_deg.csv", ppi_graph, n_permutations=100, step_label="full")
print(f"Full differentiation step done in {(time.time() - start)/60:.2f} minutes.")


PPI graph has 18451 nodes and 345547 edges.

Identifying key genes for step 1 ...

Original DEG list for step 1: 70 genes
DEGs in PPI network for step 1: 70 genes
10/100 permutations complete.
20/100 permutations complete.
30/100 permutations complete.
40/100 permutations complete.
50/100 permutations complete.
60/100 permutations complete.
70/100 permutations complete.
80/100 permutations complete.
90/100 permutations complete.
100/100 permutations complete.
22 genes have at least one p-value ≤ 0.05 for step 1
Significant genes for step 1 (p ≤ 0.05):
['RHOA', 'FN1', 'SRGAP2', 'DPYSL3', 'PTPRJ', 'ROBO1', 'TGFBR1', 'TGFBR3', 'VCL', 'RECK', 'CAPZB', 'PLXNA2', 'APP', 'CD44', 'LAMC1', 'PTEN', 'IGF1', 'NRP1', 'SEMA3A', 'FOS', 'JUN', 'PPARG']
Step 1 done in 23.12 minutes.

Identifying key genes for step 2 ...

Original DEG list for step 2: 99 genes
DEGs in PPI network for step 2: 98 genes
10/100 permutations complete.
20/100 permutations complete.
30/100 permutations complete.
40/100 permuta

In [5]:
# Save CSV files with only the key genes

# Load the significant genes for each step
step1_key_genes = pd.read_csv("../results/humanPVATsn/network_analysis/step1_significant_genes.csv")
step2_key_genes = pd.read_csv("../results/humanPVATsn/network_analysis/step2_significant_genes.csv")
step3_key_genes = pd.read_csv("../results/humanPVATsn/network_analysis/step3_significant_genes.csv")
full_key_genes = pd.read_csv("../results/humanPVATsn/network_analysis/full_significant_genes.csv")

# Reformat each dataframe to have a single column 'Gene' without the header
def reformat_key_genes(df):
    return df[['Gene']]
step1_key_genes = reformat_key_genes(step1_key_genes)
step2_key_genes = reformat_key_genes(step2_key_genes)
step3_key_genes = reformat_key_genes(step3_key_genes)
full_key_genes = reformat_key_genes(full_key_genes)

# Save the reformatted key genes to CSV files
step1_key_genes.to_csv("../results/humanPVATsn/network_analysis/step1_key_genes_only.csv", index=False, header=False)
step2_key_genes.to_csv("../results/humanPVATsn/network_analysis/step2_key_genes_only.csv", index=False, header=False)
step3_key_genes.to_csv("../results/humanPVATsn/network_analysis/step3_key_genes_only.csv", index=False, header=False)
full_key_genes.to_csv("../results/humanPVATsn/network_analysis/full_key_genes_only.csv", index=False, header=False)